In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

## Carga de datos

In [27]:
# Cargar datos
try:
    df = pd.read_csv('../../data/data/application_train.csv')
    print(f"Dataset cargado: {len(df)} registros, {len(df.columns)} columnas")
except FileNotFoundError:
    print("Archivo no encontrado. Creando datos sintéticos para pruebas.")
    np.random.seed(42)
    n_records = 1000
    
    df = pd.DataFrame({
        'SK_ID_CURR': range(1, n_records + 1),
        'TARGET': np.random.choice([0, 1], n_records, p=[0.92, 0.08]),
        'DAYS_BIRTH': np.random.randint(-25000, -6000, n_records),
        'DAYS_EMPLOYED': np.random.randint(-15000, 0, n_records),
        'DAYS_REGISTRATION': np.random.randint(-6000, 0, n_records),
        'DAYS_ID_PUBLISH': np.random.randint(-7000, 0, n_records),
        'NAME_INCOME_TYPE': np.random.choice(['Working', 'Commercial associate', 'Student', 'Pensioner', 'State servant'], n_records),
        'AMT_INCOME_TOTAL': np.random.lognormal(11, 1, n_records),
        'AMT_CREDIT': np.random.lognormal(12, 0.5, n_records),
        'CNT_CHILDREN': np.random.choice([0, 1, 2, 3, 4, 5], n_records, p=[0.4, 0.3, 0.2, 0.07, 0.02, 0.01]),
        'NAME_FAMILY_STATUS': np.random.choice(['Single / not married', 'Married', 'Civil marriage', 'Widow', 'Separated'], n_records),
        'NAME_EDUCATION_TYPE': np.random.choice(['Secondary / secondary special', 'Higher education', 'Incomplete higher', 'Lower secondary'], n_records),
        'ORGANIZATION_TYPE': np.random.choice(['Business Entity Type 3', 'School', 'Government', 'Bank', 'Medicine', 'Construction'], n_records),
        'OCCUPATION_TYPE': np.random.choice(['Laborers', 'Core staff', 'Sales staff', 'Managers', 'Drivers'], n_records),
        'HOUR_APPR_PROCESS_START': np.random.randint(0, 24, n_records)
    })
    print(f"Dataset sintético creado: {len(df)} registros")

print(f"\nColumnas disponibles: {list(df.columns)}")

Dataset cargado: 307511 registros, 122 columnas

Columnas disponibles: ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', '

## 1. Detección de Inconsistencias Temporales Críticas

In [28]:
def detectar_trabajadores_antes_nacer(df):
    """
    Encuentra personas que comenzaron a trabajar antes de nacer
    """
    problemas = []
    
    for _, row in df.iterrows():
        dias_nacimiento = abs(row['DAYS_BIRTH'])  # Convertir a positivo
        dias_empleo = abs(row['DAYS_EMPLOYED'])    # Convertir a positivo
        
        # Si empezó a trabajar más días atrás que su nacimiento
        if dias_empleo > dias_nacimiento:
            edad_actual = dias_nacimiento / 365
            anos_trabajando = dias_empleo / 365
            diferencia = anos_trabajando - edad_actual
            
            problemas.append({
                'SK_ID_CURR': row['SK_ID_CURR'],
                'EDAD_ACTUAL': round(edad_actual, 1),
                'ANOS_TRABAJANDO': round(anos_trabajando, 1),
                'DIFERENCIA_IMPOSIBLE': round(diferencia, 1),
                'TIPO_INGRESO': row['NAME_INCOME_TYPE'],
                'PROBLEMA': f'Trabajó {diferencia:.1f} años antes de nacer'
            })
    
    return pd.DataFrame(problemas)

# Detectar casos
casos_imposibles = detectar_trabajadores_antes_nacer(df)
print(f"Casos encontrados de personas que trabajaron antes de nacer: {len(casos_imposibles)}")

if len(casos_imposibles) > 0:
    print("\nPrimeros 10 casos más críticos:")
    top_casos = casos_imposibles.sort_values('DIFERENCIA_IMPOSIBLE', ascending=False).head(10)
    for _, caso in top_casos.iterrows():
        print(f"ID {caso['SK_ID_CURR']}: Edad {caso['EDAD_ACTUAL']} años, trabajando {caso['ANOS_TRABAJANDO']} años ({caso['PROBLEMA']})")
else:
    print("No se encontraron casos imposibles en los datos actuales")

Casos encontrados de personas que trabajaron antes de nacer: 55374

Primeros 10 casos más críticos:
ID 134473: Edad 21.1 años, trabajando 1000.7 años (Trabajó 979.6 años antes de nacer)
ID 346121: Edad 21.1 años, trabajando 1000.7 años (Trabajó 979.5 años antes de nacer)
ID 445410: Edad 21.8 años, trabajando 1000.7 años (Trabajó 978.9 años antes de nacer)
ID 134900: Edad 21.9 años, trabajando 1000.7 años (Trabajó 978.7 años antes de nacer)
ID 344627: Edad 22.1 años, trabajando 1000.7 años (Trabajó 978.6 años antes de nacer)
ID 108111: Edad 22.2 años, trabajando 1000.7 años (Trabajó 978.4 años antes de nacer)
ID 241618: Edad 22.3 años, trabajando 1000.7 años (Trabajó 978.4 años antes de nacer)
ID 212725: Edad 22.3 años, trabajando 1000.7 años (Trabajó 978.4 años antes de nacer)
ID 265549: Edad 22.3 años, trabajando 1000.7 años (Trabajó 978.3 años antes de nacer)
ID 301397: Edad 22.8 años, trabajando 1000.7 años (Trabajó 977.9 años antes de nacer)


## 2. Fechas de Registro Futuras o Imposibles

In [29]:
def detectar_fechas_futuras(df):
    """
    Encuentra fechas que están en el futuro (imposible)
    """
    problemas = []
    
    for _, row in df.iterrows():
        issues = []
        
        # Verificar días positivos (futuro)
        if row['DAYS_BIRTH'] > 0:
            issues.append(f"Nacerá en {row['DAYS_BIRTH']/365:.1f} años")
        
        if row['DAYS_EMPLOYED'] > 0:
            issues.append(f"Empezará a trabajar en {row['DAYS_EMPLOYED']/365:.1f} años")
        
        if 'DAYS_REGISTRATION' in df.columns and row['DAYS_REGISTRATION'] > 0:
            issues.append(f"Se registrará en {row['DAYS_REGISTRATION']/365:.1f} años")
        
        if issues:
            problemas.append({
                'SK_ID_CURR': row['SK_ID_CURR'],
                'PROBLEMAS': '; '.join(issues),
                'NUM_PROBLEMAS': len(issues)
            })
    
    return pd.DataFrame(problemas)

fechas_futuras = detectar_fechas_futuras(df)
print(f"Casos con fechas en el futuro: {len(fechas_futuras)}")

if len(fechas_futuras) > 0:
    print("\nCasos más problemáticos:")
    for _, caso in fechas_futuras.sort_values('NUM_PROBLEMAS', ascending=False).head(5).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['PROBLEMAS']}")

Casos con fechas en el futuro: 55374

Casos más problemáticos:
ID 456252: Empezará a trabajar en 1000.7 años
ID 100011: Empezará a trabajar en 1000.7 años
ID 100015: Empezará a trabajar en 1000.7 años
ID 100027: Empezará a trabajar en 1000.7 años
ID 100045: Empezará a trabajar en 1000.7 años


## 3. Inconsistencias en Edad vs Tipo de Ingreso

In [30]:
def detectar_estudiantes_sospechosos(df):
    """
    Encuentra estudiantes de edad muy avanzada o con ingresos muy altos
    """
    estudiantes = df[df['NAME_INCOME_TYPE'] == 'Student'].copy()
    estudiantes['EDAD'] = abs(estudiantes['DAYS_BIRTH']) / 365
    
    sospechosos = []
    
    for _, est in estudiantes.iterrows():
        problemas = []
        
        # Estudiantes mayores de 50 años
        if est['EDAD'] > 50:
            problemas.append(f"Estudiante de {est['EDAD']:.0f} años")
        
        # Ingresos muy altos para estudiante
        if est['AMT_INCOME_TOTAL'] > df['AMT_INCOME_TOTAL'].quantile(0.9):
            problemas.append(f"Ingresos ${est['AMT_INCOME_TOTAL']:,.0f} (top 10%)")
        
        if problemas:
            sospechosos.append({
                'SK_ID_CURR': est['SK_ID_CURR'],
                'EDAD': round(est['EDAD'], 1),
                'INGRESOS': est['AMT_INCOME_TOTAL'],
                'PROBLEMAS': '; '.join(problemas)
            })
    
    return pd.DataFrame(sospechosos)

def detectar_jubilados_sospechosos(df):
    """
    Encuentra jubilados muy jóvenes
    """
    jubilados = df[df['NAME_INCOME_TYPE'] == 'Pensioner'].copy()
    jubilados['EDAD'] = abs(jubilados['DAYS_BIRTH']) / 365
    
    # Jubilados menores de 50 años (sospechoso)
    jovenes = jubilados[jubilados['EDAD'] < 50]
    
    return jovenes[['SK_ID_CURR', 'EDAD', 'AMT_INCOME_TOTAL']].copy()

# Ejecutar detecciones
estudiantes_sospechosos = detectar_estudiantes_sospechosos(df)
jubilados_jovenes = detectar_jubilados_sospechosos(df)

print(f"Estudiantes sospechosos: {len(estudiantes_sospechosos)}")
print(f"Jubilados jóvenes (< 50 años): {len(jubilados_jovenes)}")

if len(estudiantes_sospechosos) > 0:
    print("\nEstudiantes más sospechosos:")
    for _, caso in estudiantes_sospechosos.head(5).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['PROBLEMAS']}")

if len(jubilados_jovenes) > 0:
    print("\nJubilados más jóvenes:")
    for _, caso in jubilados_jovenes.sort_values('EDAD').head(5).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['EDAD']:.1f} años, ingresos ${caso['AMT_INCOME_TOTAL']:,.0f}")

Estudiantes sospechosos: 6
Jubilados jóvenes (< 50 años): 2445

Estudiantes más sospechosos:
ID 111963: Estudiante de 51 años
ID 156534: Estudiante de 53 años
ID 203919: Estudiante de 51 años
ID 221688: Estudiante de 54 años
ID 309635: Estudiante de 53 años

Jubilados más jóvenes:
ID 134473.0: 21.1 años, ingresos $85,500
ID 346121.0: 21.1 años, ingresos $103,500
ID 445410.0: 21.8 años, ingresos $94,500
ID 134900.0: 21.9 años, ingresos $67,500
ID 344627.0: 22.1 años, ingresos $67,500
ID 134473.0: 21.1 años, ingresos $85,500
ID 346121.0: 21.1 años, ingresos $103,500
ID 445410.0: 21.8 años, ingresos $94,500
ID 134900.0: 21.9 años, ingresos $67,500
ID 344627.0: 22.1 años, ingresos $67,500


## 4. Inconsistencias Familiares

In [31]:
def detectar_familias_inconsistentes(df):
    """
    Encuentra inconsistencias temporales reales en estructuras familiares
    """
    problemas = []
    
    for _, row in df.iterrows():
        edad = abs(row['DAYS_BIRTH']) / 365
        hijos = row.get('CNT_CHILDREN', 0)
        estado = row.get('NAME_FAMILY_STATUS', '')
        
        issues = []
        
        # Casos realmente sospechosos: personas que se casaron antes de nacer
        # Simulamos esto con personas muy jóvenes (menos de 16) con estado civil no-soltero
        if edad < 16 and estado not in ['Single / not married', '']:
            issues.append(f"Casado/a a los {edad:.1f} años (menor de edad)")
        
        # Personas con más de 15 hijos (biológicamente muy improbable)
        if hijos > 15:
            issues.append(f"Tiene {hijos} hijos (número extremo)")
        
        # Personas menores de 10 años con hijos (imposible biológicamente)
        if edad < 10 and hijos > 0:
            issues.append(f"Tiene {hijos} hijos a los {edad:.1f} años (imposible)")
        
        # Personas menores de 14 años con más de 1 hijo
        if edad < 14 and hijos > 1:
            issues.append(f"Tiene {hijos} hijos a los {edad:.1f} años (muy improbable)")
        
        # Casos extremos: personas que reportan estar casadas pero con edad negativa calculada
        dias_birth = row['DAYS_BIRTH']
        if dias_birth > 0 and estado in ['Married', 'Civil marriage']:
            issues.append(f"Casado/a pero nacerá en {dias_birth/365:.1f} años")
        
        if issues:
            problemas.append({
                'SK_ID_CURR': row['SK_ID_CURR'],
                'EDAD': round(edad, 1),
                'HIJOS': hijos,
                'ESTADO': estado,
                'PROBLEMAS': '; '.join(issues)
            })
    
    return pd.DataFrame(problemas)

familias_inconsistentes = detectar_familias_inconsistentes(df)
print(f"Familias con inconsistencias: {len(familias_inconsistentes)}")

if len(familias_inconsistentes) > 0:
    print("\nCasos más llamativos:")
    for _, caso in familias_inconsistentes.head(10).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['PROBLEMAS']}")

Familias con inconsistencias: 2

Casos más llamativos:
ID 280108: Tiene 19 hijos (número extremo)
ID 407877: Tiene 19 hijos (número extremo)


## 5. Ocupaciones vs Organizaciones Incompatibles

In [32]:
def detectar_ocupaciones_incompatibles(df):
    """
    Encuentra combinaciones extrañas de ocupación y organización
    """
    # Definir incompatibilidades obvias
    incompatibles = {
        'School': ['Drivers', 'Construction workers'],  # Conductores en escuelas
        'Bank': ['Laborers', 'Cooking staff'],          # Obreros en bancos
        'Medicine': ['Sales staff', 'Drivers'],         # Vendedores en medicina
        'Construction': ['Core staff', 'Managers']      # Gerentes en construcción básica
    }
    
    problemas = []
    
    for org, ocupaciones_raras in incompatibles.items():
        for ocupacion in ocupaciones_raras:
            casos = df[
                (df['ORGANIZATION_TYPE'] == org) & 
                (df['OCCUPATION_TYPE'] == ocupacion)
            ]
            
            for _, caso in casos.iterrows():
                edad = abs(caso['DAYS_BIRTH']) / 365
                problemas.append({
                    'SK_ID_CURR': caso['SK_ID_CURR'],
                    'EDAD': round(edad, 1),
                    'ORGANIZACION': org,
                    'OCUPACION': ocupacion,
                    'PROBLEMA': f"{ocupacion} trabajando en {org}"
                })
    
    return pd.DataFrame(problemas)

ocupaciones_raras = detectar_ocupaciones_incompatibles(df)
print(f"Ocupaciones incompatibles detectadas: {len(ocupaciones_raras)}")

if len(ocupaciones_raras) > 0:
    print("\nCombinaciones más extrañas:")
    for _, caso in ocupaciones_raras.head(10).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['PROBLEMA']} (edad {caso['EDAD']} años)")
    
    # Contar por tipo
    print("\nResumen por tipo de incompatibilidad:")
    for problema in ocupaciones_raras['PROBLEMA'].value_counts().head(5).items():
        print(f"{problema[0]}: {problema[1]} casos")

Ocupaciones incompatibles detectadas: 1252

Combinaciones más extrañas:
ID 100664: Drivers trabajando en School (edad 55.0 años)
ID 102186: Drivers trabajando en School (edad 39.7 años)
ID 106616: Drivers trabajando en School (edad 43.8 años)
ID 117769: Drivers trabajando en School (edad 45.7 años)
ID 120919: Drivers trabajando en School (edad 44.0 años)
ID 124251: Drivers trabajando en School (edad 27.5 años)
ID 126236: Drivers trabajando en School (edad 53.7 años)
ID 128650: Drivers trabajando en School (edad 53.6 años)
ID 128901: Drivers trabajando en School (edad 31.1 años)
ID 130040: Drivers trabajando en School (edad 31.7 años)

Resumen por tipo de incompatibilidad:
Managers trabajando en Construction: 654 casos
Drivers trabajando en Medicine: 284 casos
Drivers trabajando en School: 106 casos
Sales staff trabajando en Medicine: 83 casos
Core staff trabajando en Construction: 77 casos


## 6. Horarios de Aplicación Sospechosos

In [33]:
def detectar_horarios_sospechosos(df):
    """
    Encuentra aplicaciones en horarios muy extraños
    """
    if 'HOUR_APPR_PROCESS_START' not in df.columns:
        return pd.DataFrame()
    
    # Horarios sospechosos: 1-5 AM
    madrugada = df[
        (df['HOUR_APPR_PROCESS_START'] >= 1) & 
        (df['HOUR_APPR_PROCESS_START'] <= 5)
    ].copy()
    
    madrugada['EDAD'] = abs(madrugada['DAYS_BIRTH']) / 365
    
    # Casos más sospechosos: jubilados o estudiantes aplicando de madrugada
    muy_sospechosos = madrugada[
        madrugada['NAME_INCOME_TYPE'].isin(['Student', 'Pensioner'])
    ]
    
    return madrugada[['SK_ID_CURR', 'HOUR_APPR_PROCESS_START', 'EDAD', 'NAME_INCOME_TYPE']].copy()

horarios_sospechosos = detectar_horarios_sospechosos(df)
print(f"Aplicaciones en horarios sospechosos (1-5 AM): {len(horarios_sospechosos)}")

if len(horarios_sospechosos) > 0:
    print("\nPrimeros casos:")
    for _, caso in horarios_sospechosos.head(10).iterrows():
        print(f"ID {caso['SK_ID_CURR']}: {caso['NAME_INCOME_TYPE']} aplicó a las {caso['HOUR_APPR_PROCESS_START']}:00 (edad {caso['EDAD']:.0f})")
    
    # Distribución por hora
    print("\nDistribución por hora de la madrugada:")
    for hora in sorted(horarios_sospechosos['HOUR_APPR_PROCESS_START'].value_counts().items()):
        print(f"  {hora[0]}:00 - {hora[1]} aplicaciones")

Aplicaciones en horarios sospechosos (1-5 AM): 7349

Primeros casos:
ID 100061: Commercial associate aplicó a las 3:00 (edad 29)
ID 100159: Commercial associate aplicó a las 3:00 (edad 40)
ID 100188: Working aplicó a las 4:00 (edad 42)
ID 100286: Working aplicó a las 5:00 (edad 49)
ID 100356: Working aplicó a las 3:00 (edad 58)
ID 100365: Pensioner aplicó a las 4:00 (edad 56)
ID 100415: Working aplicó a las 5:00 (edad 37)
ID 100469: Commercial associate aplicó a las 4:00 (edad 29)
ID 100492: State servant aplicó a las 5:00 (edad 35)
ID 100506: Working aplicó a las 4:00 (edad 40)

Distribución por hora de la madrugada:
  1:00 - 86 aplicaciones
  2:00 - 305 aplicaciones
  3:00 - 1230 aplicaciones
  4:00 - 2090 aplicaciones
  5:00 - 3638 aplicaciones


## Análisis de Propietarios de Carros Sospechosos

In [35]:
def detectar_carros_sospechosos(df):
    """Detecta personas que tenían carro antes de nacer o en edades muy jóvenes"""
    casos_sospechosos = []
    
    # Buscar columnas relacionadas con carros
    cols_auto = [col for col in df.columns if 'CAR' in col.upper() or 'AUTO' in col.upper()]
    
    if not cols_auto:
        return pd.DataFrame()
    
    # Si tenemos información de edad del carro
    if 'OWN_CAR_AGE' in df.columns and 'DAYS_BIRTH' in df.columns:
        # Filtrar solo personas que tienen carro (edad del carro no es nula)
        tiene_carro = df[df['OWN_CAR_AGE'].notna()]
        
        if len(tiene_carro) > 0:
            # Calcular edad de la persona y edad cuando obtuvo el carro
            tiene_carro = tiene_carro.copy()
            tiene_carro['edad_persona'] = (-tiene_carro['DAYS_BIRTH']) / 365
            tiene_carro['edad_cuando_obtuvo_carro'] = tiene_carro['edad_persona'] - tiene_carro['OWN_CAR_AGE']
            
            # Caso 1: Personas que obtuvieron carro antes de nacer (edad negativa)
            carro_antes_nacer = tiene_carro[tiene_carro['edad_cuando_obtuvo_carro'] < 0]
            
            for _, row in carro_antes_nacer.head(15).iterrows():
                casos_sospechosos.append({
                    'id': row.name,
                    'problema': 'Carro antes de nacer',
                    'edad_actual': round(row['edad_persona'], 1),
                    'edad_carro': row['OWN_CAR_AGE'],
                    'edad_cuando_obtuvo': round(row['edad_cuando_obtuvo_carro'], 1),
                    'descripcion': f'Obtuvo carro {abs(round(row["edad_cuando_obtuvo_carro"], 1))} años antes de nacer'
                })
            
            # Caso 2: Niños muy pequeños con carro (menos de 16 años cuando lo obtuvieron)
            ninos_con_carro = tiene_carro[
                (tiene_carro['edad_cuando_obtuvo_carro'] > 0) &  # No antes de nacer
                (tiene_carro['edad_cuando_obtuvo_carro'] < 16)    # Muy jóvenes
            ]
            
            for _, row in ninos_con_carro.head(20).iterrows():
                casos_sospechosos.append({
                    'id': row.name,
                    'problema': 'Carro en edad muy temprana',
                    'edad_actual': round(row['edad_persona'], 1),
                    'edad_carro': row['OWN_CAR_AGE'],
                    'edad_cuando_obtuvo': round(row['edad_cuando_obtuvo_carro'], 1),
                    'descripcion': f'Obtuvo carro a los {round(row["edad_cuando_obtuvo_carro"], 1)} años'
                })
            
            # Caso 3: Carros muy viejos (más de 50 años)
            carros_antiguos = tiene_carro[tiene_carro['OWN_CAR_AGE'] > 50]
            
            for _, row in carros_antiguos.head(10).iterrows():
                casos_sospechosos.append({
                    'id': row.name,
                    'problema': 'Carro extremadamente viejo',
                    'edad_actual': round(row['edad_persona'], 1),
                    'edad_carro': row['OWN_CAR_AGE'],
                    'edad_cuando_obtuvo': round(row['edad_cuando_obtuvo_carro'], 1),
                    'descripcion': f'Carro de {row["OWN_CAR_AGE"]} años de antigüedad'
                })
    
    # Si solo tenemos FLAG_OWN_CAR pero no edad
    elif 'FLAG_OWN_CAR' in df.columns and 'DAYS_BIRTH' in df.columns:
        tiene_carro_flag = df[(df['FLAG_OWN_CAR'] == 1) | (df['FLAG_OWN_CAR'] == 'Y')]
        
        if len(tiene_carro_flag) > 0:
            tiene_carro_flag = tiene_carro_flag.copy()
            tiene_carro_flag['edad_persona'] = (-tiene_carro_flag['DAYS_BIRTH']) / 365
            
            # Niños muy jóvenes con carro
            ninos_con_carro = tiene_carro_flag[tiene_carro_flag['edad_persona'] < 18]
            
            for _, row in ninos_con_carro.head(15).iterrows():
                casos_sospechosos.append({
                    'id': row.name,
                    'problema': 'Menor de edad con carro',
                    'edad_actual': round(row['edad_persona'], 1),
                    'descripcion': f'Menor de {round(row["edad_persona"], 1)} años con carro'
                })
    
    if casos_sospechosos:
        return pd.DataFrame(casos_sospechosos)
    else:
        return pd.DataFrame()

# Ejecutar análisis de carros sospechosos
carros_sospechosos = detectar_carros_sospechosos(df)

In [38]:
# Analizar casos de carros sospechosos
if len(carros_sospechosos) > 0:
    print(f"Total de casos sospechosos de carros encontrados: {len(carros_sospechosos)}")
    
    # Filtrar los casos más problemáticos
    casos_criticos = carros_sospechosos[carros_sospechosos['problema'] == 'Carro antes de nacer']
    casos_ninos = carros_sospechosos[carros_sospechosos['problema'] == 'Carro en edad muy temprana']
    casos_antiguos = carros_sospechosos[carros_sospechosos['problema'] == 'Carro extremadamente viejo']
    
    # Mostrar casos críticos
    if len(casos_criticos) > 0:
        print(f"\nCasos críticos - Carros antes de nacer: {len(casos_criticos)}")
        for i, (_, caso) in enumerate(casos_criticos.head(5).iterrows()):
            print(f"{i+1}. ID {caso['id']}: {caso['descripcion']}")
    
    # Mostrar casos de niños
    if len(casos_ninos) > 0:
        print(f"\nCasos preocupantes - Carros en edad temprana: {len(casos_ninos)}")
        for i, (_, caso) in enumerate(casos_ninos.head(5).iterrows()):
            print(f"{i+1}. ID {caso['id']}: {caso['descripcion']}")
    
    # Mostrar carros muy antiguos
    if len(casos_antiguos) > 0:
        print(f"\nCarros extremadamente viejos: {len(casos_antiguos)}")
        for i, (_, caso) in enumerate(casos_antiguos.head(3).iterrows()):
            print(f"{i+1}. ID {caso['id']}: {caso['descripcion']}")
    
    # Estadísticas generales
    if 'edad_cuando_obtuvo' in carros_sospechosos.columns:
        edad_promedio_carro = carros_sospechosos['edad_cuando_obtuvo'].mean()
else:
    print("No se detectaron casos sospechosos relacionados con carros en este dataset.")

Total de casos sospechosos de carros encontrados: 45

Casos críticos - Carros antes de nacer: 15
1. ID 85: Obtuvo carro 21.4 años antes de nacer
2. ID 134: Obtuvo carro 26.0 años antes de nacer
3. ID 272: Obtuvo carro 37.0 años antes de nacer
4. ID 369: Obtuvo carro 28.4 años antes de nacer
5. ID 415: Obtuvo carro 38.0 años antes de nacer

Casos preocupantes - Carros en edad temprana: 20
1. ID 13: Obtuvo carro a los 15.6 años
2. ID 15: Obtuvo carro a los 6.9 años
3. ID 69: Obtuvo carro a los 9.7 años
4. ID 92: Obtuvo carro a los 8.4 años
5. ID 93: Obtuvo carro a los 11.7 años

Carros extremadamente viejos: 10
1. ID 85: Carro de 65.0 años de antigüedad
2. ID 134: Carro de 65.0 años de antigüedad
3. ID 237: Carro de 64.0 años de antigüedad
